In [14]:
import re
import heapq 
import copy
import sys
import math

sys.setrecursionlimit(100000)

In [15]:
threeDimMap = {
    (0,0) : (1,1,2),
    (30,180) : (1, 0.67, 1.67),
    (60,180) : (1, 0.33, 1.33),
    (90,180) : (1, 0, 1),
    (120,180) : (1, 0.33, 0.67),
    (150,180) : (1, 0.67, 0.33),
    (180,180) : (1, 1, 0),
    (150,0) : (1, 1.33, 0.33),
    (120,0) : (1, 1.67, 0.67),
    (90,0) : (1, 2, 1),
    (60,0) : (1, 1.67, 1.33),
    (30,0) : (1, 1.33, 1.67),
    
    (30,270) : (1.33, 1, 1.67),
    (60,270) : (1.67, 1, 1.33),
    (90,270) : (2, 1, 1),
    (120,270) : (1.67, 1, 0.67),
    (150,270) : (1.33, 1, 0.33),
    (150,90) : (0.67, 1, 0.33),
    (120,90) : (0.33, 1, 0.67),
    (90,90) : (0, 1, 1),
    (60,90) : (0.33, 1, 1.33),
    (30,90) : (0.67, 1, 1.67),
    
    (90,330) : (1.33, 1.67, 1),
    (90,300) : (1.67, 1.33, 1),
    (90,240) : (1.67, 0.67, 1),
    (90,210) : (1.33, 0.33, 1),
    (90,150) : (0.67, 0.33, 1),
    (90,120) : (0.33, 0.67, 1),
    (90,60) : (0.33, 1.33, 1),
    (90,30) : (0.67, 1.67, 1)
    
}

def heuristic(X, Y):
    x = threeDimMap[X]
    y = threeDimMap[Y]
    distance = 2*math.sqrt(sum([(a - b) ** 2 for a, b in zip(x, y)]))
    return distance


In [16]:
heuristic((150,90), (60,180))

2.4962371682193982

In [17]:
axis0_180 = {}
axis90_270 = {}
equator = {}

def populateAxes(tileNums):
    actualPos = (int(tileNums[0]), int(tileNums[1]))
    truePos = (int(tileNums[2]), int(tileNums[3]))
                
    #populate equator
    if (actualPos[0] == 90):
        equator[actualPos] = truePos
        
    #populate axis0-180
    if (actualPos[1] == 0):
        axis0_180[actualPos] = truePos
    if (actualPos[1] == 180):
        axis0_180[actualPos] = truePos
                
    #populate axis90-270
    if(actualPos[1] == 90):
        axis90_270[actualPos] = truePos
    if(actualPos[1] == 270):
        axis90_270[actualPos] = truePos
    if(actualPos[0] == 0 & actualPos[1] == 0):
        axis90_270[actualPos] = truePos
    if(actualPos[0] == 180 & actualPos[1] == 180):
        axis90_270[actualPos] = truePos
    
def readState(filename):
    file1 = open(filename,"r")
    lines = file1.readlines()
    #print (lines[1:31])
    #print("\n")
        
    for line in lines[1:31]:
        tileNums = re.findall(r'\d+', line)
        #print(tileNums)
        if(len(tileNums)>4):
            #print(tileNums[2:6])
            populateAxes(tileNums[2:6])
        else:
            #print(tileNums)
            populateAxes(tileNums)

readState("Puzzle-14.mb")

In [18]:
class GlobeState:
    
    def __init__(self, axis0_180, axis90_270, equator, g):
        self.axis0_180 = axis0_180
        self.axis90_270 = axis90_270
        self.equator = equator
        self.g = g
        
    #rotate clockwise axis 0-180
    def RC_axis0_180(self):
        newD = {}
        
        newD[(30,180)] = self.axis0_180[(0,0)]
        
        for i in [60, 90, 120, 150, 180]:
            newD[(i, 180)] = self.axis0_180[(i-30,180)]
        
        newD[(150,0)] = self.axis0_180[(180,180)]
        
        for i in [120, 90, 60, 30, 0]:
            newD[(i, 0)] = self.axis0_180[(i+30,0)]
        
        #swaps in axis90_270    
        self.axis90_270[(0,0)] = self.axis0_180[(30,0)]
        self.axis90_270[(180,180)] = self.axis0_180[150,180]
        
        #swaps in equator
        self.equator[(90,0)] = self.axis0_180[120,0]
        self.equator[(90,180)] = self.axis0_180[60,180]
        
        #new axis0_180
        self.axis0_180 = newD
    
    def RCC_axis0_180(self):
        newD = {}
        
        newD[(0,0)] = self.axis0_180[(30,180)]
        
        for i in [30, 60, 90, 120, 150]:
            newD[(i,0)] = self.axis0_180[(i-30,0)]
        
        newD[(180, 180)] = self.axis0_180[(150,0)]
        
        for i in [150, 120, 90, 60, 30]:
            newD[(i,180)] = self.axis0_180[(i+30, 180)]
            
        #swaps in axis90_270
        self.axis90_270[(0,0)] = self.axis0_180[(30,180)]
        self.axis90_270[(180, 180)] = self.axis0_180[(150, 0)]
        
        #swaps in equator
        self.equator[(90,0)] = self.axis0_180[(60,0)]
        self.equator[(90,180)] = self.axis0_180[(120, 180)]
        
        #new axis0_180
        self.axis0_180 = newD
    
    def RC_axis90_270(self):
        newD = {}
        
        newD[(30,270)] = self.axis90_270[(0,0)]
        
        for i in [60, 90, 120, 150]:
            newD[(i,270)] = self.axis90_270[(i-30, 270)]
        
        newD[(180,180)] = self.axis90_270[(150,270)]
        newD[(150, 90)] = self.axis90_270[(180,180)]
        
        for i in [120, 90, 60, 30]:
            newD[(i, 90)] = self.axis90_270[(i+30,90)]
        
        newD[(0,0)] = self.axis90_270[(30,90)]
        
        #swaps in axis0_180
        self.axis0_180[(0,0)] = self.axis90_270[(30,90)]
        self.axis0_180[(180,180)] = self.axis90_270[(150,270)]
        
        #swaps in equator
        self.equator[(90,90)] = self.axis90_270[(120,90)]
        self.equator[(90,270)] = self.axis90_270[(60,270)]
        
        #new axis90_270
        self.axis90_270 = newD
    
    def RCC_axis90_270(self):
        newD = {}
        
        newD[(0,0)] = self.axis90_270[(30, 270)]
        newD[(30,90)] = self.axis90_270[(0,0)]
        
        for i in [60, 90, 120, 150]:
            newD[(i,90)] = self.axis90_270[(i-30,90)]
            
        newD[(180,180)] = self.axis90_270[(150,90)]
        newD[(150, 270)] = self.axis90_270[(180,180)]
        
        for i in [120, 90, 60, 30]:
            newD[(i,270)] = self.axis90_270[(i+30, 270)]
        
        #sawps in axis0_180
        self.axis0_180[(0,0)] = self.axis90_270[(30, 270)]
        self.axis0_180[(180,180)] = self.axis90_270[(150, 90)]
        
        #swaps in equator
        self.equator[(90,90)] = self.axis90_270[(60,90)]
        self.equator[(90,270)] = self.axis90_270[(120,270)]
        
        #new axis90_270
        self.axis90_270 = newD
    
    def RC_equator(self):
        newD = {}
        
        newD[(90,330)] = self.equator[(90,0)]
        
        for i in [300, 270, 240, 210, 180, 150, 120, 90, 60, 30, 0]:
            newD[(90, i)] = self.equator[(90, i+30)]
        
        #swaps in axis0_180
        self.axis0_180[(90,0)] = self.equator[(90, 30)]
        self.axis0_180[(90,180)] = self.equator[(90, 210)]
        
        #swaps in axis90_270
        self.axis90_270[(90, 90)] = self.equator[(90, 120)]
        self.axis90_270[(90, 270)] = self.equator[(90, 300)]
        
        #new equator
        self.equator = newD
        
    def RCC_equator(self):
        newD = {}
        
        newD[(90,0)] = self.equator[(90,330)]
        
        for i in [30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330]:
            newD[(90, i)] = self.equator[(90, i-30)]
        
        #swaps in axis0_180
        self.axis0_180[(90,0)] = self.equator[(90, 330)]
        self.axis0_180[(90,180)] = self.equator[(90, 150)]
        
        #swaps in axis90_270
        self.axis90_270[(90, 90)] = self.equator[(90, 60)]
        self.axis90_270[(90, 270)] = self.equator[(90, 240)]
        
        #new equator
        self.equator = newD
        
    def getCurrentHeuristic(self):
        H = 0
        
        for key in self.axis0_180.keys():
            H = H + heuristic(key, self.axis0_180[key])
        
        for key in self.axis90_270.keys() - self.axis0_180.keys():
            H = H + heuristic(key, self.axis90_270[key])
        
        for key in self.equator.keys() - self.axis90_270.keys() - self.axis0_180.keys():
            H = H + heuristic(key, self.equator[key])
        
        return H/12
    
    
    def isSolved(self):
        flag = True
        for key in self.axis0_180.keys():
            if (key != self.axis0_180[key]):
                flag = False
        for key in self.axis90_270.keys():
            if (key != self.axis90_270[key]):
                flag = False
        for key in self.equator.keys():
            if (key != self.equator[key]):
                flag = False
        
        return flag
    
    

In [19]:
gs = GlobeState(axis0_180, axis90_270, equator, 0)
#gs.RCC_axis0_180()

gs.isSolved()

False

In [20]:
def copyState(state):
    newState = GlobeState(state.axis0_180, state.axis90_270, state.equator, state.g)
    return newState

In [21]:
def Astar(n):
    #initialize open and closed lists
    openList=set()
    closedList=set()
    
    #k=0
    #add start state to openList
    openList.add(gs)
    
    while(len(openList)!=0):
        #print(closedList)
        #k=k+1
        #find state with min f val
        minVal=10000000000
        for state in openList:
            if(state.getCurrentHeuristic() + state.g < minVal):
                minVal = state.getCurrentHeuristic() + state.g
                print (minVal)
                q = state
        #remove q from openList
        openList.remove(q)
        
        q.g=q.g+1
        
        #create successors
        gs1 = copy.deepcopy(q)
        gs2 = copy.deepcopy(q)
        gs3 = copy.deepcopy(q)
        gs4 = copy.deepcopy(q)
        gs5 = copy.deepcopy(q)
        gs6 = copy.deepcopy(q)
        
        gs1.RC_axis0_180()
        gs2.RCC_axis0_180()
        gs3.RC_axis90_270()
        gs4.RCC_axis90_270()
        gs5.RC_equator()
        gs6.RCC_equator()
        
        #for each succesor, if successor in either list and f val in listed item is greater, skip this successor.
        #else add it to openList
        for state in [gs1, gs2, gs3, gs4, gs5, gs6]:
            if(state.isSolved()):
                return state
            
            state.g = q.g
            f = state.g + state.getCurrentHeuristic()
            
            flag = True 
            
            if(state.g > n):
                flag=False
            
            for stateX in openList:
                if (stateX.axis0_180 == state.axis0_180) & (stateX.axis90_270 == state.axis90_270) & (stateX.equator == state.equator):
                    #if stateX.getCurrentHeuristic() + stateX.g < f:
                    flag = False
            
            for stateX in closedList:
                if (stateX.axis0_180 == state.axis0_180) & (stateX.axis90_270 == state.axis90_270) & (stateX.equator == state.equator):
                    #if stateX.getCurrentHeuristic() + stateX.g < f:
                    flag = False
            
            if(flag):
                
                openList.add(state)
            
        closedList.add(q)
        
        
        

In [ ]:
gh = Astar(100)

5.094654378671106
6.3380339547027384
5.701709529401931
7.202836002888327
6.3380339547027384
5.957355250405322
5.890136974033866
7.202836002888327
6.697472853013616
6.3380339547027384
5.957355250405322
7.116897192442125
6.988467005339664
6.50294085619658
6.229807438593972
6.225848362747798
7.116897192442125
6.957337279147056
6.50294085619658
6.229807438593972
7.116897192442125
6.7814975965703335
6.50294085619658
6.397956315364156
6.3380339547027384
7.116897192442125
6.7814975965703335
6.50294085619658
6.397956315364156
7.7802800256334566
7.360855686204947
7.116897192442125
6.7814975965703335
6.50294085619658
7.7802800256334566
7.360855686204947
7.116897192442125
6.7814975965703335
6.719973892997188
6.719973892997187
6.665422928825955
7.7802800256334566
7.360855686204947
7.116897192442125
6.7814975965703335
6.719973892997188
6.719973892997187
6.697472853013616
7.7802800256334566
7.360855686204947
7.116897192442125
6.7814975965703335
6.719973892997188
6.719973892997187
7.7802800256334566


8.300351591442306
8.171371295543175
8.146119338082563
8.06055788145326
7.903390923281789
8.871588245920353
8.635768582567714
8.593464063490103
8.4347725282841
8.434010367606255
8.41121026461208
8.300351591442306
8.171371295543175
8.146119338082563
8.06055788145326
8.041150251571882
7.942675578640771
7.905701619848607
8.871588245920353
8.635768582567714
8.593464063490103
8.4347725282841
8.434010367606255
8.41121026461208
8.300351591442306
8.171371295543175
8.146119338082563
8.06055788145326
8.041150251571882
7.942675578640771
7.915050731327713
7.909305092420095
8.871588245920353
8.635768582567714
8.593464063490103
8.4347725282841
8.434010367606255
8.41121026461208
8.300351591442306
8.171371295543175
8.146119338082563
8.06055788145326
8.041150251571882
7.942675578640771
7.915050731327713
8.871588245920353
8.635768582567714
8.593464063490103
8.4347725282841
8.434010367606255
8.41121026461208
8.300351591442306
8.171371295543175
8.146119338082563
8.06055788145326
8.041150251571882
7.9426755

8.871588245920353
8.635768582567714
8.593464063490103
8.4347725282841
8.434010367606255
8.41121026461208
8.300351591442306
8.171371295543175
8.146119338082563
8.140289384186584
8.135716584196771
8.126283433794764
8.871588245920353
8.635768582567714
8.593464063490103
8.4347725282841
8.434010367606255
8.41121026461208
8.300351591442306
8.171371295543175
8.146119338082563
8.140289384186584
8.135716584196771
8.871588245920353
8.635768582567714
8.593464063490103
8.4347725282841
8.434010367606255
8.41121026461208
8.300351591442306
8.171371295543175
8.146119338082563
8.140289384186584
8.871588245920353
8.635768582567714
8.593464063490103
8.4347725282841
8.434010367606255
8.41121026461208
8.300351591442306
8.171371295543175
8.146119338082563
8.871588245920353
8.635768582567714
8.593464063490103
8.4347725282841
8.434010367606255
8.41121026461208
8.300351591442306
8.171371295543175
8.162343013720612
8.162224154572202
8.155007649127946
8.871588245920353
8.635768582567714
8.593464063490103
8.43477

8.871588245920353
8.635768582567714
8.593464063490103
8.4347725282841
8.434010367606255
8.41121026461208
8.3848958416873
8.370657764680654
8.345269184190643
8.345013804585925
8.338739151465337
8.334121430897074
8.871588245920353
8.635768582567714
8.593464063490103
8.4347725282841
8.434010367606255
8.41121026461208
8.3848958416873
8.370657764680654
8.345269184190643
8.345013804585925
8.338739151465337
8.338305848951045
8.871588245920353
8.635768582567714
8.593464063490103
8.4347725282841
8.434010367606255
8.41121026461208
8.3848958416873
8.370657764680654
8.345269184190643
8.345013804585925
8.338739151465337
8.871588245920353
8.635768582567714
8.593464063490103
8.4347725282841
8.434010367606255
8.41121026461208
8.3848958416873
8.370657764680654
8.345269184190643
8.345013804585925
8.871588245920353
8.635768582567714
8.593464063490103
8.4347725282841
8.434010367606255
8.41121026461208
8.3848958416873
8.370657764680654
8.345269184190643
8.871588245920353
8.635768582567714
8.593464063490103

9.32770756605218
8.871588245920353
8.635768582567714
8.593464063490103
8.592501542350199
8.56029573171369
8.552696268136149
8.545154324052877
9.32770756605218
8.871588245920353
8.635768582567714
8.593464063490103
8.592501542350199
8.56029573171369
8.552696268136149
8.551033171782452
9.32770756605218
8.871588245920353
8.635768582567714
8.593464063490103
8.592501542350199
8.56029573171369
8.552696268136149
8.551463711106013
9.32770756605218
8.871588245920353
8.635768582567714
8.593464063490103
8.592501542350199
8.56029573171369
8.552696268136149
9.32770756605218
8.871588245920353
8.635768582567714
8.593464063490103
8.592501542350199
8.56029573171369
8.554969834829432
9.32770756605218
8.871588245920353
8.635768582567714
8.593464063490103
8.592501542350199
8.56029573171369
9.32770756605218
8.871588245920353
8.635768582567714
8.593464063490103
8.592501542350199
8.584239098973097
8.578455578315483
8.56029573171369
9.32770756605218
8.871588245920353
8.635768582567714
8.593464063490103
8.59250

...

10.70481714385803
10.70434201177435
10.902502249588991
10.880179752299053
10.878455092157205
10.841497943051909
10.777506662311563
10.719224257906149
10.716814073073142
10.71144651454398
10.709883282869843
10.707655983568632
10.706596663643491
10.70528734866738
10.70481714385803
10.704377421904283
10.704347539458098
10.902502249588991
10.880179752299053
10.878455092157205
10.841497943051909
10.777506662311563
10.719224257906149
10.716814073073142
10.71144651454398
10.709883282869843
10.707655983568632
10.706596663643491
10.70528734866738
10.70481714385803
10.704377421904283
10.902502249588991
10.880179752299053
10.878455092157205
10.841497943051909
10.777506662311563
10.719224257906149
10.716814073073142
10.71144651454398
10.709883282869843
10.707655983568632
10.706596663643491
10.70528734866738
10.70481714385803
10.704438828439415
10.902502249588991
10.880179752299053
10.878455092157205
10.841497943051909
10.777506662311563
10.719224257906149
10.716814073073142
10.71144651454398
10.70

10.706881190435872
10.706871038219376
10.902502249588991
10.880179752299053
10.878455092157205
10.841497943051909
10.777506662311563
10.719224257906149
10.716814073073142
10.71144651454398
10.709883282869843
10.707655983568632
10.707638616845909
10.707621380788872
10.707128440834268
10.706881190435872
10.902502249588991
10.880179752299053
10.878455092157205
10.841497943051909
10.777506662311563
10.719224257906149
10.716814073073142
10.71144651454398
10.709883282869843
10.707655983568632
10.707638616845909
10.707621380788872
10.707128440834268
10.70694813271012
10.902502249588991
10.880179752299053
10.878455092157205
10.841497943051909
10.777506662311563
10.719224257906149
10.716814073073142
10.71144651454398
10.709883282869843
10.707655983568632
10.707638616845909
10.707621380788872
10.707128440834268
10.902502249588991
10.880179752299053
10.878455092157205
10.841497943051909
10.777506662311563
10.719224257906149
10.716814073073142
10.71144651454398
10.709883282869843
10.70765598356863

10.710397820431407
10.710305933197535
10.710279143838601
10.710254166427863
10.902502249588991
10.880179752299053
10.878455092157205
10.841497943051909
10.777506662311563
10.719224257906149
10.716814073073142
10.71144651454398
10.711186570294732
10.710397820431407
10.710305933197535
10.710279143838601
10.902502249588991
10.880179752299053
10.878455092157205
10.841497943051909
10.777506662311563
10.719224257906149
10.716814073073142
10.71144651454398
10.711186570294732
10.710397820431407
10.710305933197535
10.902502249588991
10.880179752299053
10.878455092157205
10.841497943051909
10.777506662311563
10.719224257906149
10.716814073073142
10.71144651454398
10.711186570294732
10.710397820431407
10.902502249588991
10.880179752299053
10.878455092157205
10.841497943051909
10.777506662311563
10.719224257906149
10.716814073073142
10.71144651454398
10.711186570294732
10.710669714679712
10.71054314286451
10.902502249588991
10.880179752299053
10.878455092157205
10.841497943051909
10.77750666231156

10.902502249588991
10.880179752299053
10.878455092157205
10.841497943051909
10.777506662311563
10.719224257906149
10.716814073073142
10.714980417507588
10.7146336586503
10.714503048687993
10.714503048687991
10.71440111443069
10.714163453840953
10.71406457499519
10.902502249588991
10.880179752299053
10.878455092157205
10.841497943051909
10.777506662311563
10.719224257906149
10.716814073073142
10.714980417507588
10.7146336586503
10.714503048687993
10.714503048687991
10.71440111443069
10.714163453840953
10.71409519913631
10.902502249588991
10.880179752299053
10.878455092157205
10.841497943051909
10.777506662311563
10.719224257906149
10.716814073073142
10.714980417507588
10.7146336586503
10.714503048687993
10.714503048687991
10.71440111443069
10.714163453840953
10.902502249588991
10.880179752299053
10.878455092157205
10.841497943051909
10.777506662311563
10.719224257906149
10.716814073073142
10.714980417507588
10.7146336586503
10.714503048687993
10.714503048687991
10.71440111443069
10.7143

10.716585694240223
10.716465896218256
10.902502249588991
10.880179752299053
10.878455092157205
10.841497943051909
10.777506662311563
10.719224257906149
10.716814073073142
10.716585694240223
10.71646967174333
10.902502249588991
10.880179752299053
10.878455092157205
10.841497943051909
10.777506662311563
10.719224257906149
10.716814073073142
10.716585694240223
10.902502249588991
10.880179752299053
10.878455092157205
10.841497943051909
10.777506662311563
10.719224257906149
10.716814073073142
10.716809065953877
10.902502249588991
10.880179752299053
10.878455092157205
10.841497943051909
10.777506662311563
10.719224257906149
10.716814073073142
10.902502249588991
10.880179752299053
10.878455092157205
10.841497943051909
10.777506662311563
10.719224257906149
10.717672670088639
10.717568339697156
10.717435792375142
10.717101073543379
10.902502249588991
10.880179752299053
10.878455092157205
10.841497943051909
10.777506662311563
10.719224257906149
10.717672670088639
10.717568339697156
10.7174357923

In [13]:
gh.axis0_180

{(30, 180): (30, 180),
 (60, 180): (60, 180),
 (90, 180): (90, 180),
 (120, 180): (120, 180),
 (150, 180): (150, 180),
 (180, 180): (180, 180),
 (150, 0): (150, 0),
 (120, 0): (120, 0),
 (90, 0): (90, 0),
 (60, 0): (60, 0),
 (30, 0): (30, 0),
 (0, 0): (0, 0)}

In [2]:
int("inf")

ValueError: invalid literal for int() with base 10: 'inf'

In [7]:
def Astar(gs):
    k=0;
    h = []
    heapq.heappush(h, (gs.getCurrentHeuristic(), gs))
    
    while(h[0][1].isSolved() == False):    
        gs = heapq.heappop(h)[1]
        k=k+1
        print(k)
        gs1 = copy.copy(gs)
        gs2 = copy.copy(gs)
        gs3 = copy.copy(gs)
        gs4 = copy.copy(gs)
        gs5 = copy.copy(gs)
        gs6 = copy.copy(gs)
        
        gs1.RC_axis0_180()
        gs2.RCC_axis0_180()
        gs3.RC_axis90_270()
        gs4.RCC_axis90_270()
        gs5.RC_equator()
        gs6.RCC_equator()
             
        a = gs1.getCurrentHeuristic()+k
        b = gs2.getCurrentHeuristic()+k
        c = gs3.getCurrentHeuristic()+k
        d = gs4.getCurrentHeuristic()+k
        e = gs5.getCurrentHeuristic()+k
        f = gs6.getCurrentHeuristic()+k
        
        if ((a,gs1) not in h):
            heapq.heappush(h, (a, gs1))
        if ((b,gs2) not in h):
            heapq.heappush(h, (b, gs2))
        if ((c,gs3) not in h):
            heapq.heappush(h, (c, gs3))
        if ((d,gs4) not in h):
            heapq.heappush(h, (d, gs4))
        if ((e,gs5) not in h):
            heapq.heappush(h, (e, gs5))
        if ((f,gs6) not in h):
            heapq.heappush(h, (f, gs6))
        #print(type(d))
    return k

In [ ]:
axis0_180Base=[(0, 0),(30, 0),(60, 0),(90, 0),(120, 0),(150, 0),(180, 180),(150, 180),(120, 180),(90, 180),(60, 180),(30, 180)]

axis90_270Base=[(0, 0),(30, 90),(60, 90),(90, 90),(120, 90),(150, 90),(180, 180),(150, 270),(120, 270),(90, 270),(60, 270),(30, 270)]

equatorBase = [(90,0),(90,30),(90,60),(90,90),(90,120),(90,150),(90,180),(90,210),(90,240),(90,270),(90,300),(90,330)]

def heuristic(X, Y):
    if (X in axis0_180Base) & (Y in axis0_180Base):
        indexX = axis0_180Base.index(X)
        indexY = axis0_180Base.index(Y)
        steps = abs(indexX - indexY)
        if (steps> 6):
            k = steps - 6
            steps = 6 - k
        
        return steps
    
    elif (X in axis90_270Base) & (Y in axis90_270Base):
        indexX = axis90_270Base.index(X)
        indexY = axis90_270Base.index(Y)
        steps = abs(indexX - indexY)
        if (steps> 6):
            k = steps - 6
            steps = 6 - k
        
        return steps
    
    elif (X in equatorBase) & (Y in equatorBase):
        indexX = equatorBase.index(X)
        indexY = equatorBase.index(Y)
        steps = abs(indexX - indexY)
        if (steps> 6):
            k = steps - 6
            steps = 6 - k
        
        return steps
    else:
        return 0
    